In [1]:
import pyodbc
import requests
from datetime import datetime
import pandas as pd
import socket
import uuid




def logtoVortex(logtimestr, feedname, referencedata, reference2data, durationinms, parentId, success, responsemessage):

    VORTEXENDPOINT = "http://vortex.int.e360power.com/api/vortex/send"

    computername = socket.gethostname()

    headers = {"Content-Type": "application/json", "X2UserId": "Voltage", "X2CorrelationId": parentId}

    data = {
    "id": 0,
    "logTime": logtimestr,
    "appname": feedname,
    "itemname": "LoadData",
    "reference": referencedata,
    "reference2": reference2data,
    "isSuccess" : success,
    "responseMessage": responsemessage,
    "userName": "savolt",
    "machineName": computername,
    "duration": int(durationinms),
    "parentid": parentId,
    "count" : 1,
    "systemName": "Voltage"
    }

    response = requests.post(VORTEXENDPOINT, headers=headers, json=data)

In [2]:
query_str = '''
exec UpdateNewHvarByPg;
exec UpdateNewHvarBySg;
exec UpdateNewHvarBySgCg;
exec UpdateNewHvarByPgCg;

declare @last_bus_day date = cast(DATEADD(DAY, CASE (DATEPART(WEEKDAY, GETDATE()) + @@DATEFIRST) % 7 
                        WHEN 1 THEN -2 
                        WHEN 2 THEN -3 
                        ELSE -1 
                    END, DATEDIFF(DAY, 0, GETDATE())) as date);
                    
insert into mmbtu_by_company select * from get_mmbtu_by_company(@last_bus_day)

insert into detail_hvar_company_percentiles select * from compute_detail_hvar_company_percentiles(cast(getdate() as date))
'''

try:
    cnxn = pyodbc.connect(
            r'DRIVER={ODBC Driver 17 for SQL Server};'
            r'SERVER=e360-db01;'
            r'DATABASE=Voltage;'
            r'Trusted_Connection=yes;'
        )
    
    starttime = datetime.now()
    feedname = "Running Ken's Stored Procedures and Table Function Loaders" 
    parentId = uuid.uuid4().hex
    logtime = datetime.now()
    logtimestr = logtime.strftime("%Y-%m-%dT%H:%M:%S")
    responsemessage = "Action Complete"
    
    cursql = cnxn.cursor()
    cursql.execute(query_str)
    cnxn.commit()
    
    cursql.close()
    cnxn.close()
    
    end = datetime.now()
    
    success = 1
except Exception as e:
    success = 0
    responsemessage = str(e)
finally:
    endtime = datetime.now()
    delta = endtime - starttime
    durationinms = delta.total_seconds()
    referencedata = "none"
    reference2data = "none"
    logtoVortex(logtimestr, feedname, referencedata, reference2data, durationinms, parentId, success, responsemessage)        

In [3]:
# def e360_holidays():
#     import holidays
#     import numpy as np
#     from dateutil.easter import easter
#     from pandas.tseries.offsets import BDay

#     focus_years = [2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034]

#     us_holidays = holidays.UnitedStates(years=focus_years,
#                                         observed=True)

#     for year in focus_years:
#         us_holidays.update({pd.to_datetime(easter(year)-BDay(1)).date(): "Good Friday"})

#     focus_holidays = [holiday for holiday in us_holidays.items()]

#     e360_holidays = []
#     for index, holiday in enumerate(focus_holidays):
#         if 'Veterans' in holiday[1] or 'Columbus' in holiday[1]:
#             continue
#         else:
#             e360_holidays.append(holiday[0])

#     office_closed_days = np.busdaycalendar(holidays=e360_holidays)

#     return office_closed_days

In [4]:
# target_dates = pd.bdate_range('01/01/2023', '08/31/2023', freq='C', holidays = e360_holidays().holidays).date
# target_dates


In [5]:
# for date in target_dates:
#     query_str = f'''
#     --insert into detail_hvar_company_percentiles select * from compute_detail_hvar_company_percentiles('{date}')
#     insert into mmbtu_by_company select * from get_mmbtu_by_company('{date}')
#     '''

#     try:
#         cnxn = pyodbc.connect(
#                 r'DRIVER={ODBC Driver 17 for SQL Server};'
#                 r'SERVER=e360-db01;'
#                 r'DATABASE=Voltage;'
#                 r'Trusted_Connection=yes;'
#             )

#         starttime = datetime.now()
#         feedname = "Create base table for New Detail VaR Percentiles by Company" 
#         parentId = uuid.uuid4().hex
#         logtime = datetime.now()
#         logtimestr = logtime.strftime("%Y-%m-%dT%H:%M:%S")
#         responsemessage = "Action Complete"

#         cursql = cnxn.cursor()
#         cursql.execute(query_str)
#         cnxn.commit()

#         cursql.close()
#         cnxn.close()

#         end = datetime.now()

#         success = 1
#     except Exception as e:
#         success = 0
#         responsemessage = str(e)
#     finally:
#         endtime = datetime.now()
#         delta = endtime - starttime
#         durationinms = delta.total_seconds()
#         referencedata = "none"
#         reference2data = "none"
#         logtoVortex(logtimestr, feedname, referencedata, reference2data, durationinms, parentId, success, responsemessage) 